In [3]:
import requests
import xmltodict
import time
import pandas as pd
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [4]:
url_decide_base = "http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson"
url_decide_serviceKey = "S8%2Ftx%2BhEP7bZDZI%2By0P1ZKvPuHpx%2BVUKpt6ay8faxnxR%2FTRO9M5UAy8%2BafhJBNVzQG%2Fgwoym2S4Xbe1dUXivUw%3D%3D"

url_district_base = "http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson"
url_district_serviceKey = url_decide_serviceKey

url_seoul_vaccine = 'http://openapi.seoul.go.kr:8088/706e775a4577686136356f44596267/xml/tvCorona19VaccinestatNew/1/999/'

url_vaccine = 'https://api.odcloud.kr/api/15077756/v1/vaccine-stat?page=1&perPage=3000&serviceKey=S8%2Ftx%2BhEP7bZDZI%2By0P1ZKvPuHpx%2BVUKpt6ay8faxnxR%2FTRO9M5UAy8%2BafhJBNVzQG%2Fgwoym2S4Xbe1dUXivUw%3D%3D'

url_pages = "1000" #페이지당열갯수
url_start_date = "20200303" #시작날짜
url_end_date = datetime.today().strftime("%Y%m%d%H%M%S")[:8] #끝날짜


In [10]:
print('Getting In Data Of CORONA19 Decide')

url = url_decide_base + "?serviceKey=" + url_decide_serviceKey + "&pageNo=1&numOfRows=" + url_pages + "&startCreateDt="+ url_start_date + "&endCreateDt=" + url_end_date

req = requests.get(url).content
xmlObject = xmltodict.parse(req)
dict_data = xmlObject['response']['body']['items']['item']

dfDecide = pd.DataFrame(dict_data)

dfDecide.drop(['createDt', 'seq', 'stateTime', 'updateDt', 'resutlNegCnt'], axis=1, inplace=True)
dfDecide.sort_values(['stateDt'], ascending=False, inplace=True)
dfDecide['stateDt'] = pd.to_datetime(dfDecide['stateDt']) - pd.DateOffset(days=1)
dfDecide = dfDecide.astype({"accExamCnt":"int64", "accExamCompCnt":"int64", "careCnt":"int64", "clearCnt":"int64", "deathCnt":"int64", "decideCnt":"int64", "examCnt":"int64", "accDefRate":"float64"}).copy()

dfDecide['newDecideCnt'] = 0

for i in range(len(dfDecide)-1):
    dfDecide['newDecideCnt'][i] = int(dfDecide.iloc[i]['decideCnt']) - int(dfDecide.iloc[i+1]['decideCnt'])

dfDecide = dfDecide[['stateDt', 'newDecideCnt', 'examCnt', 'decideCnt', 'deathCnt', 'clearCnt', 'careCnt', 'accDefRate', 'accExamCnt', 'accExamCompCnt']].copy()
dfDecide.rename(columns={'stateDt':'날짜', 'newDecideCnt':'당일 확진자 수', 'examCnt':'검사 진행 수', 'decideCnt':'누적 확진자 수', 'deathCnt':'누적 사망자 수', 'clearCnt':'누적 격리 해제 수', 
                         'careCnt':'치료 중 환자 수', 'accDefRate':'누적 확진률', 'accExamCnt':'누적 검사 수', 'accExamCompCnt':'누적 검사 완료 수'}, inplace=True)

Getting In Data Of CORONA19 Decide


In [13]:
dfDecide

,날짜,당일 확진자 수,검사 진행 수,누적 확진자 수,누적 사망자 수,누적 격리 해제 수,치료 중 환자 수,누적 확진률,누적 검사 수,누적 검사 완료 수
0,2021-11-15,2125,1617465,399591,3137,364499,31955,2.680393,16525392,14907927
1,2021-11-14,2006,1635106,397466,3115,362834,31517,2.679088,16470974,14835868
2,2021-11-13,4743,1614278,395460,3103,360891,31466,2.667314,16440430,14826152
3,2021-11-11,2368,1574579,390717,3051,356615,31051,2.642399,16361030,14786451
4,2021-11-10,2520,1552667,388349,3033,355014,30302,2.631072,16312773,14760106
...,...,...,...,...,...,...,...,...,...,...
617,2020-03-06,483,19620,6767,44,115,6608,4.267543,178189,158569
618,2020-03-05,518,21832,6284,42,106,6136,4.397235,164740,142908
619,2020-03-04,438,21810,5766,35,86,5645,4.622748,146541,124731
620,2020-03-03,516,28414,5328,32,40,5256,4.919986,136707,108293


In [14]:
print('Getting In Data Of CORONA19 District')

url_district= url_district_base + "?serviceKey=" + url_district_serviceKey + "&pageNo=1&numOfRows=" + url_pages + "&startCreateDt="+ url_start_date + "&endCreateDt=" + url_end_date

req = requests.get(url_district).content

xmlObject = xmltodict.parse(req)
dict_data = xmlObject['response']['body']['items']['item']

dfDistrict = pd.DataFrame(dict_data)
dfDistrict = dfDistrict[(dfDistrict['gubun'] != '합계') & (dfDistrict['gubun'] != '검역')].reset_index(drop=True).copy()

tempDf = pd.DataFrame(columns=['stateDt', 'Seoul', 'Busan', 'Daegu', 'Incheon', 'Gwangju', 'Daejeon', 'Ulsan', 'Sejong', 'Gyeonggi-do', 'Gangwon-do', 'Chungcheongbuk-do', 'Chungcheongnam-do' ,'Jeollabuk-do' ,'Jeollanam-do' ,'Gyeongsangbuk-do' ,'Gyeongsangnam-do' ,'Jeju'])

for i in range(0, len(dfDistrict), 17):
    tempDistrict = dfDistrict.iloc[i:i+17]
    temp = tempDistrict[['incDec', 'stdDay']].transpose()
    yearData = int(temp[i]['stdDay'].split(' ')[0][:-1])
    monthData = int(temp[i]['stdDay'].split(' ')[1][:-1])
    dayData = int(temp[i]['stdDay'].split(' ')[2][:-1])

    temp.columns = dfDistrict['gubunEn'][:17]
    temp['stateDt'] = pd.to_datetime(datetime(yearData, monthData, dayData).strftime("%Y-%m-%d")) - pd.DateOffset(days=1)
    temp.head(1)
    temp = temp.head(1)[['stateDt', 'Seoul', 'Busan', 'Daegu', 'Incheon', 'Gwangju', 'Daejeon', 'Ulsan', 'Sejong', 'Gyeonggi-do', 'Gangwon-do', 'Chungcheongbuk-do', 'Chungcheongnam-do' ,'Jeollabuk-do' ,'Jeollanam-do' ,'Gyeongsangbuk-do' ,'Gyeongsangnam-do' ,'Jeju']].copy()

    tempDf = tempDf.append(temp)

tempDf.reset_index(drop=True, inplace=True)
tempDf = tempDf.astype('int64')
tempDf = tempDf.astype({"stateDt":"datetime64[ns]"})

tempDf.rename(columns={'stateDt':'날짜', 'Seoul':'서울', 'Busan':'부산', 'Daegu':'대구', 'Incheon':'인천', 'Gwangju':'광주', 'Daejeon':'대전', 'Ulsan':'울산', 'Sejong':'세종',
                           'Gyeonggi-do':'경기', 'Gangwon-do':'강원', 'Chungcheongbuk-do':'충북', 'Chungcheongnam-do':'충남', 'Jeollabuk-do':'전북',
                           'Jeollanam-do':'전남', 'Gyeongsangbuk-do':'경북', 'Gyeongsangnam-do':'경남', 'Jeju':'제주'}, inplace=True)


Getting In Data Of CORONA19 District


In [15]:
print('Getting In Data Of CORONA19 District')

url_district= url_district_base + "?serviceKey=" + url_district_serviceKey + "&pageNo=1&numOfRows=" + url_pages + "&startCreateDt="+ url_start_date + "&endCreateDt=" + url_end_date

req = requests.get(url_district).content

xmlObject = xmltodict.parse(req)
dict_data = xmlObject['response']['body']['items']['item']

dfDistrict = pd.DataFrame(dict_data)
dfDistrict = dfDistrict[(dfDistrict['gubun'] != '합계') & (dfDistrict['gubun'] != '검역')].reset_index(drop=True).copy()

tempDf = pd.DataFrame(columns=['stateDt', 'Seoul', 'Busan', 'Daegu', 'Incheon', 'Gwangju', 'Daejeon', 'Ulsan', 'Sejong', 'Gyeonggi-do', 'Gangwon-do', 'Chungcheongbuk-do', 'Chungcheongnam-do' ,'Jeollabuk-do' ,'Jeollanam-do' ,'Gyeongsangbuk-do' ,'Gyeongsangnam-do' ,'Jeju'])


Getting In Data Of CORONA19 District


In [16]:
dfDistrict

,createDt,deathCnt,defCnt,gubun,gubunCn,gubunEn,incDec,isolClearCnt,isolIngCnt,localOccCnt,overFlowCnt,qurRate,seq,stdDay,updateDt
0,2021-11-16 08:46:07.652,7,3327,제주,济州,Jeju,43,3140,180,43,0,493,13327,2021년 11월 16일 00시,null
1,2021-11-16 08:46:07.652,46,13905,경남,庆南,Gyeongsangnam-do,45,13127,732,45,0,416,13326,2021년 11월 16일 00시,null
2,2021-11-16 08:46:07.652,98,10378,경북,庆北,Gyeongsangbuk-do,9,9767,513,9,0,393,13325,2021년 11월 16일 00시,null
3,2021-11-16 08:46:07.651,25,3889,전남,全南,Jeollanam-do,29,3630,234,29,0,210,13324,2021년 11월 16일 00시,null
4,2021-11-16 08:46:07.651,69,5748,전북,全北,Jeollabuk-do,56,5271,408,55,1,319,13323,2021년 11월 16일 00시,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10586,2020-03-03 09:00:00.000,0,7,전북,全北,Jeollabuk-do,1,1,6,1,0,0.39,823,2020년 03월 03일 00시,2021-09-16 10:11:33.897
10587,2020-03-03 09:00:00.000,0,4,전남,全南,Jeollanam-do,0,1,3,0,0,0.21,822,2020년 03월 03일 00시,2021-09-16 10:11:33.897
10588,2020-03-03 09:00:00.000,7,687,경북,庆北,Gyeongsangbuk-do,59,2,678,59,0,25.80,821,2020년 03월 03일 00시,2021-09-16 10:11:33.897
10589,2020-03-03 09:00:00.000,0,63,경남,庆南,Gyeongsangnam-do,1,0,63,1,0,1.87,820,2020년 03월 03일 00시,2021-09-16 10:11:33.897


In [17]:
dfDistrict.iloc[i:i+17]

,createDt,deathCnt,defCnt,gubun,gubunCn,gubunEn,incDec,isolClearCnt,isolIngCnt,localOccCnt,overFlowCnt,qurRate,seq,stdDay,updateDt
10574,2020-03-03 09:00:00.000,0,98,서울,首尔,Seoul,7,13,85,7,0,1.01,835,2020년 03월 03일 00시,2021-09-16 10:11:33.897
10575,2020-03-03 09:00:00.000,0,87,부산,釜山,Busan,2,0,87,2,0,2.55,834,2020년 03월 03일 00시,2021-09-16 10:11:33.897
10576,2020-03-03 09:00:00.000,20,3604,대구,大邱,Daegu,520,2,3582,520,0,147.92,833,2020년 03월 03일 00시,2021-09-16 10:11:33.897
10577,2020-03-03 09:00:00.000,0,7,인천,仁川,Incheon,0,1,6,0,0,0.24,832,2020년 03월 03일 00시,2021-09-16 10:11:33.897
10578,2020-03-03 09:00:00.000,0,11,광주,光州,Gwangju,2,2,9,1,1,0.76,831,2020년 03월 03일 00시,2021-09-16 10:11:33.897
10579,2020-03-03 09:00:00.000,0,14,대전,大田,Daejeon,0,0,14,0,0,0.95,830,2020년 03월 03일 00시,2021-09-16 10:11:33.897
10580,2020-03-03 09:00:00.000,0,20,울산,蔚山,Ulsan,0,0,20,0,0,1.74,829,2020년 03월 03일 00시,2021-09-16 10:11:33.897
10581,2020-03-03 09:00:00.000,0,1,세종,世宗,Sejong,0,0,1,0,0,0.29,828,2020년 03월 03일 00시,2021-09-16 10:11:33.897
10582,2020-03-03 09:00:00.000,1,94,경기,京畿,Gyeonggi-do,2,12,81,2,0,0.71,827,2020년 03월 03일 00시,2021-09-16 10:11:33.897
10583,2020-03-03 09:00:00.000,0,20,강원,江原,Gangwon-do,1,0,20,1,0,1.30,826,2020년 03월 03일 00시,2021-09-16 10:11:33.897


In [18]:
tempDistrict[['incDec', 'stdDay']].transpose()

,10574,10575,10576,10577,10578,10579,10580,10581,10582,10583,10584,10585,10586,10587,10588,10589,10590
incDec,7,2,520,0,2,0,0,0,2,1,0,4,1,0,59,1,1
stdDay,2020년 03월 03일 00시,2020년 03월 03일 00시,2020년 03월 03일 00시,2020년 03월 03일 00시,2020년 03월 03일 00시,2020년 03월 03일 00시,2020년 03월 03일 00시,2020년 03월 03일 00시,2020년 03월 03일 00시,2020년 03월 03일 00시,2020년 03월 03일 00시,2020년 03월 03일 00시,2020년 03월 03일 00시,2020년 03월 03일 00시,2020년 03월 03일 00시,2020년 03월 03일 00시,2020년 03월 03일 00시


In [24]:
for i in range(0, len(dfDistrict), 17):
    tempDistrict = dfDistrict.iloc[i:i+17]
    temp = tempDistrict[['incDec', 'stdDay']].transpose()
    yearData = int(temp[i]['stdDay'].split(' ')[0][:-1])
    monthData = int(temp[i]['stdDay'].split(' ')[1][:-1])
    dayData = int(temp[i]['stdDay'].split(' ')[2][:-1])

In [21]:
yearData

2020

In [23]:
monthData

3

In [25]:
dayData

3